In [1]:
import requests
from openai import OpenAI
from dotenv import dotenv_values
from bs4 import BeautifulSoup
from googleapiclient.discovery import build  

config = dotenv_values("../.env") 

num_results = 5

open_ai_key = config['OPEN_AI_API_KEY']
GOOGLE_API_KEY = config['GOOGLE_API_KEY']
SEARCH_ENGINE_ID = config['SEARCH_ENGINE_ID']

client = OpenAI(api_key=open_ai_key)
service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)

In [2]:
original_query = "what investing options are offered by etrade"

In [3]:
chat_completion = client.chat.completions.create(
    messages=[
            {"role": "system", "content": "Rephrase the user's query for search engine optimization."},
            {"role": "user", "content": original_query},
        ],
    model="gpt-4o",
)

rephrased_query = chat_completion.choices[0].message.content
rephrased_query

'What investment choices are available through E*TRADE?'

In [4]:
result = service.cse().list(q=rephrased_query, cx=SEARCH_ENGINE_ID, num=num_results).execute()
result_items = result.get('items', [])

result_items[0]

{'kind': 'customsearch#result',
 'title': 'Investment Choices | Understanding Types of Investments | E*TRADE',
 'htmlTitle': '<b>Investment Choices</b> | Understanding Types of <b>Investments</b> | <b>E*TRADE</b>',
 'link': 'https://us.etrade.com/what-we-offer/investment-choices',
 'displayLink': 'us.etrade.com',
 'snippet': 'E*TRADE offers a full range of investing products, including stocks, ETFs, mutual funds, futures, and options. Learn what they are and how to buy them.',
 'htmlSnippet': '<b>E*TRADE</b> offers a full range of <b>investing</b> products, including stocks, ETFs, mutual funds, futures, and <b>options</b>. Learn what they are and how to buy them.',
 'formattedUrl': 'https://us.etrade.com/what-we-offer/investment-choices',
 'htmlFormattedUrl': 'https://us.<b>etrade</b>.com/what-we-offer/<b>investment</b>-<b>choices</b>',
 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5DX-k0rgXmN40s90cTJ6nvFZAgqn4PWIn0QZj2wKoK5eFl_vjzxEi_r4

In [5]:
url = result_items[0]['link']
url

'https://us.etrade.com/what-we-offer/investment-choices'

In [6]:
response = requests.get(url)
html_doc = response.content
soup = BeautifulSoup(html_doc, 'html.parser')
raw_text_response = soup.get_text()

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
            {"role": "system", "content": f"Refine the following text and extract only relevant information related to the following query: {rephrased_query}"},
            {"role": "user", "content": raw_text_response},
        ],
    model="gpt-4o-mini",
)

relevant_information = chat_completion.choices[0].message.content

In [8]:
print(relevant_information)

E*TRADE offers a diverse range of investment choices, including:

1. **Stocks**: Investors can buy shares, representing ownership in companies, which can be traded throughout the day.

2. **Options**: Contracts that give the right to buy or sell a stock at a set price before a specified expiration date.

3. **Futures**: Agreements to buy or sell a security at a predetermined price on or before a future date.

4. **ETFs (Exchange-Traded Funds)**: Diversified baskets of investments that can be traded like stocks throughout the day.

5. **Mutual Funds**: Investment vehicles composed of a diversified portfolio of stocks and/or bonds, priced once at the end of the trading day.

6. **Bonds and CDs (Certificates of Deposit)**: Fixed-income securities where investors lend money to institutions in exchange for interest payments and returned principal at maturity.

7. **Prebuilt Portfolios**: Automated investment options that involve managed portfolios.

8. **IPO (Initial Public Offerings)/New I